#Deep Learning Based Recommendation Systems

#Import statements

In [ ]:
!pip install recommenders[gpu] -f https://download.pytorch.org/whl/cu111/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cu111/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.0/356.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 38.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 106.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 66.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 114.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 919.8/919.8 kB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# %load_ext autoreload
# %autoreload 2

In [ ]:

import sys
import pandas as pd
import tensorflow as tf

tf.get_logger().setLevel('ERROR')
from recommenders.utils.timer import Timer
from recommenders.models.ncf.ncf_singlenode import NCF
from recommenders.models.ncf.dataset import Dataset as NCFDataset

from recommenders.utils.notebook_utils import is_jupyter
from recommenders.datasets.python_splitters import python_chrono_split,python_stratified_split
from recommenders.evaluation.python_evaluation import (rmse, mae, rsquared,exp_var, map_at_k,\
                                                       ndcg_at_k, precision_at_k,recall_at_k, get_top_k_items)

In [ ]:
print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
Pandas version: 2.0.3
Tensorflow version: 2.15.0


#Data Loading

In [ ]:
# read Record dataset
record_df = pd.read_excel("Rec_sys_data.xlsx")
#read Customer Dataset
customer_df = pd.read_excel("Rec_sys_data.xlsx", sheet_name = 'customer')
# read product dataset
prod_df = pd.read_excel("Rec_sys_data.xlsx", sheet_name = 'product')

In [ ]:
record_df.head()

,InvoiceNo,StockCode,Quantity,InvoiceDate,DeliveryDate,Discount%,ShipMode,ShippingCost,CustomerID
0,536365,84029E,6,2010-12-01 08:26:00,2010-12-02 08:26:00,0.20,ExpressAir,30.12,17850
1,536365,71053,6,2010-12-01 08:26:00,2010-12-02 08:26:00,0.21,ExpressAir,30.12,17850
2,536365,21730,6,2010-12-01 08:26:00,2010-12-03 08:26:00,0.56,Regular Air,15.22,17850
3,536365,84406B,8,2010-12-01 08:26:00,2010-12-03 08:26:00,0.30,Regular Air,15.22,17850
4,536365,22752,2,2010-12-01 08:26:00,2010-12-04 08:26:00,0.57,Delivery Truck,5.81,17850


In [ ]:
customer_df.head()

,CustomerID,Gender,Age,Income,Zipcode,Customer Segment
0,13089,male,53,High,8625,Small Business
1,15810,female,22,Low,87797,Small Business
2,15556,female,29,High,29257,Corporate
3,13137,male,29,Medium,97818,Middle class
4,16241,male,36,Low,79200,Small Business


In [ ]:
prod_df.head()

,StockCode,Product Name,Description,Category,Brand,Unit Price
0,22629,Ganma Superheroes Ordinary Life Case For Samsu...,"New unique design, great gift.High quality pla...",Cell Phones|Cellphone Accessories|Cases & Prot...,Ganma,13.99
1,21238,Eye Buy Express Prescription Glasses Mens Wome...,Rounded rectangular cat-eye reading glasses. T...,Health|Home Health Care|Daily Living Aids,Eye Buy Express,19.22
2,22181,MightySkins Skin Decal Wrap Compatible with Ni...,Each Nintendo 2DS kit is printed with super-hi...,Video Games|Video Game Accessories|Accessories...,Mightyskins,14.99
3,84879,Mediven Sheer and Soft 15-20 mmHg Thigh w/ Lac...,The sheerest compression stocking in its class...,Health|Medicine Cabinet|Braces & Supports,Medi,62.38
4,84836,Stupell Industries Chevron Initial Wall D cor,Features: -Made in the USA. -Sawtooth hanger o...,Home Improvement|Paint|Wall Decals|All Wall De...,Stupell Industries,35.99


#Data Preprocessing

In [ ]:
#selecting columns
df = record_df[['CustomerID','StockCode','Quantity','DeliveryDate']]
#dropping the StockCodes (item ids) that are string for this experiment, as NCF only takes integer ids
df["StockCode"] = df["StockCode"].apply(lambda x: pd.to_numeric(x,errors='coerce')).dropna()

<ipython-input-8-eb1b5f0b9311>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["StockCode"] = df["StockCode"].apply(lambda x: pd.to_numeric(x,errors='coerce')).dropna()


In [ ]:
# dropping nulls
df = df.dropna()
print(df.shape)

(246706, 4)


In [ ]:
df

,CustomerID,StockCode,Quantity,DeliveryDate
1,17850,71053.0,6,2010-12-02 08:26:00
2,17850,21730.0,6,2010-12-03 08:26:00
4,17850,22752.0,2,2010-12-04 08:26:00
7,17850,22633.0,6,2010-12-04 08:28:00
8,17850,22632.0,6,2010-12-03 08:28:00
...,...,...,...,...
272399,15249,23399.0,12,2011-10-08 11:37:00
272400,15249,22727.0,4,2011-10-08 11:37:00
272401,15249,23434.0,12,2011-10-08 11:37:00
272402,15249,23340.0,12,2011-10-07 11:37:00


In [ ]:
# Let’s rename a few of the column names.
#header=["userID", "itemID", "rating", "timestamp"]
df = df.rename(columns={'CustomerID':"userID",'StockCode':"itemID",'Quantity':"rating",'DeliveryDate':"timestamp"})


In [ ]:
#Change the user_id and item_id datatypes to an integer since that is the
# required format for NCF.
df["userID"] = df["userID"].astype(int)
df["itemID"] = df["itemID"].astype(int)

#DAta Splitting

In [ ]:
# Splitting the data using the Spark chronological splitter provided in the utilities.
train, test = python_chrono_split(df, 0.75)

In [ ]:
# Save the train and test data into two separate files

train_file = "./train.csv"
test_file = "./test.csv"
train.to_csv(train_file, index=False)
test.to_csv(test_file, index=False)

#Modeling

Neural Collaborative Filtering (NCF):

In [ ]:
# Defining the network parameters

In [ ]:
# top k items to recommend
TOP_K = 10
# Model parameters
EPOCHS = 50
BATCH_SIZE = 256
SEED = 42


In [ ]:
#preparing the data
data = NCFDataset(train_file=train_file, test_file=test_file, seed=SEED)

100%|██████████| 3498/3498 [01:18<00:00, 44.38it/s]


In [ ]:
# training NCF model
model = NCF (
    #
    n_users=data.n_users,
    n_items=data.n_items,
    model_type="NeuMF",
    n_factors=4,
    layer_sizes=[16,8,4],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=1e-3,
    verbose=10,
    seed=SEED
)


/usr/local/lib/python3.10/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1697: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


This code snippet is for training a Neural Collaborative Filtering (NCF) model.

- `n_users`: The total number of users in the dataset.
- `n_items`: The total number of items (or products) in the dataset.
- `model_type`: Specifies the type of NCF model to use. In this case, it's set to "NeuMF", which stands for Neural Matrix Factorization.
- `n_factors`: The number of latent factors used to represent users and items in the model. It determines the dimensionality of the user and item embeddings.
- `layer_sizes`: A list specifying the sizes of the hidden layers in the neural network part of the model. Here, it defines a neural network with three hidden layers having 16, 8, and 4 units, respectively.
- `n_epochs`: The number of epochs (iterations) to train the model.
- `batch_size`: The number of samples per batch to use during training.
- `learning_rate`: The learning rate used by the optimizer during training. It controls the step size taken during optimization.
- `verbose`: Controls the verbosity level of the training process. In this case, it's set to 10, meaning that training progress will be printed every 10 epochs.
- `seed`: The random seed used for reproducibility of the training process.

The `NCF` class is initialized with these parameters to create the NCF model. Once initialized, the model can be trained on the dataset using methods like `fit()` or `train()`.

In [ ]:
model

In [ ]:
#adding timer for training.
with Timer() as train_time:
  model.fit(data)
print("Took {} seconds for training.".format(train_time))

Took 1143.7530 seconds for training.


In [ ]:
with Timer() as test_time:
    users, items, preds = [], [], []
    item = list(train.itemID.unique())
    for user in train.userID.unique():
        user = [user] * len(item)
        users.extend(user)
        items.extend(item)
        preds.extend(list(model.predict(user, item, is_list=True)))

    all_predictions = pd.DataFrame(data={"userID": users, "itemID":items, "prediction":preds})

    merged = pd.merge(train, all_predictions, on=["userID", "itemID"], how="outer")
    all_predictions = merged[merged.rating.isnull()].drop('rating', axis=1)

print("Took {} seconds for prediction.".format(test_time))

Took 47.4868 seconds for prediction.


In [ ]:
# Evaluate model
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')


MAP:	0.033975
NDCG:	0.076923
Precision@K:	0.052687
Recall@K:	0.061639


#Recommendation

In [22]:
df_order = pd.read_excel('Rec_sys_data.xlsx', 'order')
df_customer = pd.read_excel('Rec_sys_data.xlsx', 'customer')
df_product = pd.read_excel('Rec_sys_data.xlsx', 'product')

In [23]:
#select columns
all_predictions = all_predictions[['userID','itemID','prediction']]
# rename columns
all_predictions = all_predictions.rename(columns={"userID":'CustomerID',\
                                                  "itemID":'StockCode',"rating":'Quantity','prediction':'probability'})

In [24]:
def recommend_product(customer_id):
    print(" \n---------- Top 5 Bought StockCodes -----------\n")
    print(df_order[df_order['CustomerID'] == customer_id][['CustomerID', 'StockCode', 'Quantity']].nlargest(5, 'Quantity'))
    top_5_bought = df_order[df_order['CustomerID'] == customer_id][['CustomerID', 'StockCode', 'Quantity']].nlargest(5, 'Quantity')
    print('\n-------Product Name of bought StockCodes ------\n')
    print(df_product[df_product.StockCode.isin(top_5_bought.StockCode)]['Product Name'])
    print("\n --------- Top 5 Recommendations ------------ \n")
    print(all_predictions[all_predictions['CustomerID'] == customer_id].nlargest(5, 'probability'))
    recommend = all_predictions[all_predictions['CustomerID'] == customer_id].nlargest(5, 'probability')
    print('\n-------Product Name of Recommendations ------\n')
    print(df_product[df_product.StockCode.isin(recommend.StockCode)]['Product Name'])


In [25]:
recommend_product(13137)

 
---------- Top 5 Bought StockCodes -----------

        CustomerID StockCode  Quantity
234414       13137     84077        48
234443       13137     23321        13
50797        13137     21985        12
234404       13137     22296        12
234418       13137     22297        12

-------Product Name of bought StockCodes ------

70      MightySkins Skin Decal Wrap Compatible with Li...
490           Window Tint Film Mitsubishi (all doors) DIY
694     Harriton Men's Paradise Short-Sleeve Performan...
1065    MightySkins Skin For Samsung Galaxy J3 (2016),...
1339    MightySkins Skin Decal Wrap Compatible with Le...
Name: Product Name, dtype: object

 --------- Top 5 Recommendations ------------ 

         CustomerID  StockCode  probability
1512164       13137      20723     0.958084
1511921       13137      22383     0.957656
1512450       13137      21932     0.952142
1511817       13137      22666     0.944453
1511681       13137      20719     0.935833

-------Product Name of Recom

In [26]:
recommend_product(15127)

 
---------- Top 5 Bought StockCodes -----------

        CustomerID StockCode  Quantity
272296       15127     23263        48
272287       15127     23354        24
272288       15127     22813        24
272289       15127     23096        24
272285       15127     21181        12

-------Product Name of bought StockCodes ------

13                billyboards Porcelain School Chalkboard
374     MightySkins Protective Vinyl Skin Decal for Po...
923     Zoan Synchrony Duo Sport Electric Snow Helmet ...
952     MightySkins Skin Decal Wrap Compatible with Sm...
1576    EMPIRE KLIX Klutch Designer Wallet Case for LG G2
Name: Product Name, dtype: object

 --------- Top 5 Recommendations ------------ 

         CustomerID  StockCode  probability
4692923       15127      23353     0.963401
4692694       15127      22818     0.956075
4693392       15127      23503     0.908095
4692707       15127      22816     0.890611
4693372       15127      23355     0.888135

-------Product Name of Recom